In [2]:
import pandas as pd

In [3]:
reviews = pd.read_csv("reviews.csv")
listings = pd.read_csv("listings.csv")


In [4]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,90676,619503,2011-10-11,1160153,Kelly,Audra was very welcoming and seems like she'd ...
1,90676,683039,2011-11-01,1290336,Dave,Audra was a very helpful and gracious host. T...
2,90676,972073,2012-03-05,1160684,Seth,Audra has a very beautiful home in a convenien...
3,90676,1067300,2012-04-01,1290336,Dave,Ggreat house and a great location and the host...
4,90676,1083843,2012-04-04,1919810,Josh,Audra and Lacey were such wonderful hosts and ...


In [9]:
for i in listings.columns:
    print(i)

id
listing_url
scrape_id
last_scraped
source
name
description
neighborhood_overview
picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
latitude
longitude
property_type
room_type
accommodates
bathrooms
bathrooms_text
bedrooms
beds
amenities
price
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_nights_avg_ntm
maximum_nights_avg_ntm
calendar_updated
has_availability
availability_30
availability_60
availability_90
availability_365
calendar_last_scraped
number_of_reviews
number_of_reviews_ltm
number_of_reviews_l30d
first_review
last_review
review_scores_rating
review_scores_accuracy
re